In [1]:
import gzip
from collections import defaultdict
import string

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)



In [2]:
#for data prepare
with_cid = list()
for l in readGz('train.json.gz'):
    if 'categoryID' in l.keys():
        with_cid.append(l)
train = with_cid[:len(with_cid)/2]
valid = with_cid[len(with_cid)/2:]
punctuation = set(string.punctuation)

In [3]:
# find most freq words in train with given ID
wordcount = defaultdict(int)
for t in train:    
    r = ''.join([c for c in t['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        wordcount[w] += 1
mostfreq = [(wordcount[w], w) for w in wordcount]
mostfreq.sort()
mostfreq.reverse()
print mostfreq[500]
        

(345, u'plates')


In [4]:
# find most freq cid in train with given ID
cidcount = defaultdict(int)
for t in train:    
    cidcount[t['categoryID']]+=1
counts = [(cidcount[w], w) for w in cidcount]
counts.sort()
counts.reverse()
print counts

[(10482, 0), (6953, 2), (4400, 1), (3531, 6), (2466, 8), (2126, 7), (1879, 3), (1199, 4), (1127, 5), (934, 9)]


In [5]:
#separate words from each cate
cateword = defaultdict(set)
for t in train:    
    r = ''.join([c for c in t['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        cateword[int(t['categoryID'])].add(w)
amer = cateword[0]
unicatw = list()
for i in range(0,10):
    unicatw.append(cateword[i])
    for j in range(0,10):
        if not i == j:
            unicatw[i] = unicatw[i].difference(cateword[j])
print unicatw[2]

set([u'saimin', u'stuy', u'folklore', u'changboaka', u'phung', u'desirable', u'whiteandsohipsterithurts', u'authrntic', u'sichuan', u'harrowdown', u'venuebrew', u'controversial', u'sfers', u'tackiness', u'differentnot', u'laterand', u'detains', u'dinging', u'fanits', u'shiu', u'jaipur', u'punjab', u'poorest', u'mantwo', u'bargainpriced\u201d', u'wizardry', u'nrv', u'dishesprimarily', u'tme', u'340', u'delecios', u'demoted', u'teaching', u'highceilinged', u'seaew', u'meadows', u'pokers', u'loungeclubbing', u'chennai', u'flushing', u'fallapart', u'gujarathi', u'appetits', u'specifications\u2014no', u'kiang', u'politei', u'fattiness', u'gimmie', u'forties', u'softened', u'watereddown', u'dynasty', u'gobbled', u'wooded', u'bewildered', u'towncar', u'crops', u'vegetablesspicy', u'sigiri', u'conveniency', u'humidity', u'kingfisher', u'notifying', u'beaucoup', u'potajanee', u'299', u'rendang', u'thaipanasian', u'tipsys', u'firstcomefirstserve', u'fattyness', u'moffet', u'karedok', u'galas', u

In [6]:
inter = cateword[0]
for i in range(1,10):
    inter = inter.intersection(cateword[i])
print len(inter)
print inter

dif = list()
for i in range(10):
    dif.append(cateword[i].difference(inter))


1423
set([u'limited', u'four', u'hanging', u'hate', u'forget', u'under', u'sorry', u'worth', u'every', u'bringing', u'school', u'frozen', u'heading', u'enjoy', u'consistent', u'bacon', u'chef', u'second', u'street', u'blue', u'cooking', u'above', u'new', u'ever', u'never', u'meh', u'here', u'100', u'dry', u'kids', u'items', u'employees', u'credit', u'smoke', u'changes', u'fantastic', u'highly', u'brought', u'guests', u'total', u'would', u'negative', u'music', u'recommend', u'type', u'tell', u'relax', u'brings', u'aware', u'glass', u'warm', u'excellent', u'hole', u'hold', u'must', u'me', u'room', u'work', u'upscale', u'my', u'give', u'foods', u'veggies', u'want', u'times', u'absolute', u'end', u'turn', u'provide', u'returning', u'how', u'hot', u'disappoint', u'beach', u'pizza', u'mess', u'after', u'wrong', u'types', u'third', u'appreciate', u'green', u'south', u'things', u'worst', u'order', u'office', u'over', u'satisfied', u'before', u'somewhere', u'highlights', u'fix', u'expectations'

In [7]:
# use valid to check acc
correct = float(0)
for v in valid:
    r = ''.join([c for c in v['reviewText'].lower() if not c in punctuation])
    vd = defaultdict(int)
    for w in r.split():
        for i in range(10):
            if w in (dif[i]):
                vd[i]+=1
    vote = [(vd[i],i) for i in range(10)]
    vote.sort()
    predict = vote[-1][1]
    if int(predict) == int(v['categoryID']):
        correct +=1
print (correct/len(valid))  
    

0.265428229529


In [50]:
#Task1 

import gzip
from collections import defaultdict
import math
import numpy
import urllib
import scipy.optimize
import random
import json
from sklearn.metrics import accuracy_score

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

def category_words(category):
  return set([word for subcategory in category for word in subcategory])

def pearson_item(i1, i2):
  R_i1_avg = numpy.mean([u['rating'] for u in U_i[i1]])
  R_i2_avg = numpy.mean([u['rating'] for u in U_i[i2]])
  R_intersection = [{'i1': u['rating'], 'i2': v['rating']} for u in U_i[i1] for v in U_i[i2] if u['userID'] == v['userID']]
  numerator = (sum([(r['i1'] - R_i1_avg) * (r['i2'] - R_i2_avg) for r in R_intersection])) * 1.0
  denominator = (sum([(r['i1'] - R_i1_avg) ** 2 for r in R_intersection]) * sum([(r['i2'] - R_i2_avg) ** 2 for r in R_intersection])) ** 0.5
  return 0.0 if denominator == 0.0 else numerator / denominator 

def pearson(user, item):
  if user not in I_u or item not in U_i:
    return True 
  for i in I_u[user]:
    if pearson_item(i['businessID'], item) > 0.01:
      return True
  return False

busCount = defaultdict(int)
userCount = defaultdict(int)
I_u = defaultdict(list)
U_i = defaultdict(list)
totalPurchases = 0


for l in readGz("train.json.gz"):
  user,bus,category = l['userID'],l['businessID'],l['categories']
  busCount[bus] += 1
  totalPurchases += 1
  # Figure out how active a user is.
  userCount[user] += 1
  if bus not in U_i:
    U_i[bus] = [l]
  U_i[bus].append(l)
  if user not in I_u:
    I_u[user] = [l]
  I_u[user].append(l)


mostPopular = [(busCount[x], x) for x in busCount]
mostPopular.sort()
mostPopular.reverse()

mostActive = [(userCount[x],x) for x in userCount]
mostActive.sort()
mostActive.reverse()

return1 = set()
count = 0
perc1 = 1#(2/3)
perc2 = 1 #(1/12)
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > (totalPurchases)*perc1: break

return2 = set()
active = 0
for ic, i in mostActive:
  active += ic
  return2.add(i)
  if active > (totalPurchases)*perc2: break

predictions = open("predictions_vs.txt", 'w')
for l in open("pairs_Visit.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if pearson(u,i):
    if i in return1 or u in return2:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

predictions.close()



In [46]:
#Task2
from sklearn import svm
from sklearn.neural_network import MLPClassifier

def feattask2(t,word500Pos):
    feat = [0]*1650
    punctuation = set(string.punctuation)
    r = ''.join([c for c in t['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        feat[word500Pos[w]] = 1
    return feat

def feattask2o(t,word500Pos):
    feat = [0]*500
    wordcount = defaultdict(int)
    punctuation = set(string.punctuation)
    r = ''.join([c for c in t['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        wordcount[w] += 1
    for w in wordcount.keys():
        if w in word500Pos.keys():
            feat[word500Pos[w]] = wordcount[w]
    return feat

'''
def train_catsvm(cat,train,c,X_train):
    y_train = [int(t['categoryID']) == int(cat) for t in train]
    clf = svm.SVC(C=c)
    clf.fit(X_train,y_train)
    return clf

def find_best_c_for_catsvm(tarin,valid,cat,X_train,X_valid):
    y_valid = [int(cat) == int(t['categoryID'])for t in valid]
     #y_valid = [ int(t['categoryID'])for t in valid]
    bestc = -1
    bestacc = -1
    for c in [0.01,0.1,1,10,100]:    
        clf = train_catsvm(cat,train,c,X_train)
        valid_pred = clf.predict(X_valid)
        acc = [(pre == val) for (pre,val) in zip(valid_pred,y_valid)]
        correct = sum (acc)
        accuracy = float(correct)/len(acc)
        if accuracy > bestacc:
            bestacc = accuracy
            bestc = c         
    return bestc
'''

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for t in train:
    r = ''.join([c for c in t['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words500 = [x[1] for x in counts[:1650]]
word500Pos = defaultdict(int)
X_train = list()
X_valid = list()
y_train = list()
y_valid_cid = list()
for i in range(len(words500)):
    word500Pos[words500[i]] = i

clf_list = list()
y_train_list = list()

for t in train:
    X_train.append(feattask2(t,word500Pos))
    y_train.append(int(t['categoryID']))

'''
for t in valid:
    X_valid.append(feattask2(t,word500Pos))
    y_valid_cid.append(int(t['categoryID']))
''' 
 
'''
new_train = train + valid
for t in new_train:
    X_train.append(feattask2(t,word500Pos))
    y_train.append(int(t['categoryID']))
'''
print 'Data ready'
'''
#best_c = [find_best_c_for_catsvm(train,valid,cat,X_train,X_valid) for cat in range(10)]
#best_catsvm = [train_catsvm(cat,train,best_c[cat],X_train)for cat in range(10)]
'''
clf = svm.LinearSVC(C=1,multi_class = 'ovr')
'''
for i in range (10):
    y_train_list.append([int(t['categoryID'])== i for t in train])
    clf_list.append(clf.fit(X_train,y_train_list[-1]))
'''
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
print 'SVMs train end'    
clf.fit(X_train,y_train)

'''
print 'SVM train 2'
valid_pred = clf.predict(X_valid)
print 'svm predict'
correct = float(0)
samples = len(y_valid_cid)
'''

'''
for i in len(y_valid):
    best_score = -1
    best_guess = -1
    for cat in range(10):
        clf = best_catsvm[cat]
        score = clf.decision_function(X_valid[i])
        if score > best_score:
            best_score = score
            best_guess = cat
    if y_valid_cid[i] == best_guess:
        correct += 1
'''

'''
for i in range(samples):
    if y_valid_cid[i] == valid_pred[i]:
        correct+=1
print correct/samples
'''

X_test = list()
predictions = open("predictions_Cat.txt", 'w')
predictions.write("userID-reviewHash,category\n")
for l in readGz("test_Category.json.gz"):
    X_test.append(feattask2(l,word500Pos))
test_pred = clf.predict(X_test)
ti = 0
for l in readGz("test_Category.json.gz"):
    '''
    bestscore = 0
    bestsvm = 0
    for i in range(10):
        score = clf_list[i].decision_function([X_test[ti]])
        if score > bestscore:
            bestsocre = score
            bestsvm = i 
    
    predictions.write(l['userID'] + '-' + l['reviewHash'] + "," + str(bestsvm) + "\n")
    '''
    predictions.write(l['userID'] + '-' + l['reviewHash'] + "," + str(test_pred[ti]) + "\n")
    ti+=1

predictions.close()


Data ready
SVMs train end


In [19]:
#count common user in train and test
userdict = defaultdict(int)
common = float(0) 
for t in train:
    userdict[t['userID']] += 1
test_sam = 0 
for l in readGz("test_Category.json.gz"):
    test_sam += 1
    if l['userID'] in userdict.keys():
        common += 1
print common/test_sam

0.8549


In [20]:
# find out each user has what category in train set
userdict = defaultdict(list)
for t in train:
    userdict[t['userID']].append(t['categoryID'])
print userdict

defaultdict(<type 'list'>, {'U600574911': [2], 'U068213890': [7], 'U774608854': [6], 'U731526909': [4, 0, 0], 'U220401668': [0, 1, 0, 1, 0, 4, 0, 9], 'U377191451': [6, 2], 'U877778143': [2], 'U264640327': [1, 6], 'U852600439': [1, 8, 0, 1], 'U143616836': [0, 6], 'U096472121': [0], 'U833075349': [2, 7, 0, 6, 9, 5, 0], 'U693165519': [2, 1], 'U156843408': [2, 6], 'U759747442': [0], 'U653154217': [2, 1], 'U282622976': [2, 8], 'U352572919': [2, 0], 'U085566721': [2, 2], 'U695537062': [2, 2], 'U133377817': [4], 'U681872193': [3, 0], 'U346784215': [2], 'U996439458': [0, 1, 2], 'U824003814': [7, 0, 5], 'U446840485': [8], 'U028170973': [1], 'U255204905': [0, 6], 'U521855745': [2, 5, 0], 'U158222645': [2], 'U008022286': [4, 6], 'U630687751': [2], 'U437473603': [1], 'U210386880': [2, 3, 2, 2, 0, 0, 4, 2, 2], 'U263662286': [2, 1], 'U425297446': [2, 4], 'U577028820': [3], 'U915631412': [0], 'U731288940': [2, 2, 2], 'U646820963': [2, 2, 6, 2, 0], 'U404440209': [7], 'U090528460': [4], 'U535544282': [